# Access S2S via Python Library

In [1]:
%%capture
%pip install "git+https://github.com/worldbank/DECAT_Space2Stats.git#subdirectory=space2stats_api/src"

In [10]:
from typing import Dict
import json

import numpy as np
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from lonboard import Map, ScatterplotLayer
from geojson_pydantic import Feature, Polygon

from space2stats import StatsTable

__Expected environment variables to be set__ from dotenv file


```.env
PGHOST=
PGPORT=
PGDATABASE=
PGUSER=
PGPASSWORD=
PGTABLENAME=space2stats
```

In [4]:
load_dotenv("../../space2stats_api/db.env")

True

In [5]:
with StatsTable.connect() as stats_table:
    fields = stats_table.fields()

fields

['sum_pop_2020',
 'ogc_fid',
 'sum_pop_f_0_2020',
 'sum_pop_f_10_2020',
 'sum_pop_f_15_2020',
 'sum_pop_f_1_2020',
 'sum_pop_f_20_2020',
 'sum_pop_f_25_2020',
 'sum_pop_f_30_2020',
 'sum_pop_f_35_2020',
 'sum_pop_f_40_2020',
 'sum_pop_f_45_2020',
 'sum_pop_f_50_2020',
 'sum_pop_f_55_2020',
 'sum_pop_f_5_2020',
 'sum_pop_f_60_2020',
 'sum_pop_f_65_2020',
 'sum_pop_f_70_2020',
 'sum_pop_f_75_2020',
 'sum_pop_f_80_2020',
 'sum_pop_m_0_2020',
 'sum_pop_m_10_2020',
 'sum_pop_m_15_2020',
 'sum_pop_m_1_2020',
 'sum_pop_m_20_2020',
 'sum_pop_m_25_2020',
 'sum_pop_m_30_2020',
 'sum_pop_m_35_2020',
 'sum_pop_m_40_2020',
 'sum_pop_m_45_2020',
 'sum_pop_m_50_2020',
 'sum_pop_m_55_2020',
 'sum_pop_m_5_2020',
 'sum_pop_m_60_2020',
 'sum_pop_m_65_2020',
 'sum_pop_m_70_2020',
 'sum_pop_m_75_2020',
 'sum_pop_m_80_2020',
 'sum_pop_m_2020',
 'sum_pop_f_2020']

In [6]:
AOIModel = Feature[Polygon, Dict]

# ~kenya
aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [33.78593974945852, 5.115816884114494],
                [33.78593974945852, -4.725410543134203],
                [41.94362577283266, -4.725410543134203],
                [41.94362577283266, 5.115816884114494],
                [33.78593974945852, 5.115816884114494]
            ]
        ]
    },
    "properties": {
        "name": "Updated AOI"
    }
}
          
  

feat = AOIModel(**aoi)

In [13]:
with StatsTable.connect() as stats_table:
    data = stats_table.summaries(aoi=feat, spatial_join_method="centroid", fields=fields, geometry="point")
    df = pd.DataFrame(data)

df.head()

,hex_id,geometry,sum_pop_2020,ogc_fid,sum_pop_f_0_2020,sum_pop_f_10_2020,sum_pop_f_15_2020,sum_pop_f_1_2020,sum_pop_f_20_2020,sum_pop_f_25_2020,...,sum_pop_m_50_2020,sum_pop_m_55_2020,sum_pop_m_5_2020,sum_pop_m_60_2020,sum_pop_m_65_2020,sum_pop_m_70_2020,sum_pop_m_75_2020,sum_pop_m_80_2020,sum_pop_m_2020,sum_pop_f_2020
0,866a4a017ffffff,"{""type"":""Point"",""coordinates"":[36.078694056627...",676.912804,6157254,8.924362,49.524105,36.898296,32.985901,28.963400,33.645473,...,6.732327,4.131749,53.380043,3.240541,2.475817,0.702667,0.399492,0.129285,336.082180,340.830624
1,866a4a01fffffff,"{""type"":""Point"",""coordinates"":[36.134817927246...",347.182722,6157255,4.515680,24.773327,19.462585,16.690670,15.832314,16.429201,...,3.445144,2.752507,25.548544,1.827761,1.524122,0.585427,0.381080,0.209030,171.759225,175.423497
2,866a4a04fffffff,"{""type"":""Point"",""coordinates"":[36.339286712001...",285.943490,6157260,3.642580,19.623219,16.698730,13.463530,14.255625,12.501910,...,2.827743,3.055085,18.765499,1.711561,1.571700,0.762171,0.533085,0.349744,140.698267,145.245222
3,866a4a057ffffff,"{""type"":""Point"",""coordinates"":[36.237081352434...",279.463912,6157261,3.560037,19.178549,16.320332,13.158442,13.932587,12.218613,...,2.763665,2.985856,18.340267,1.672776,1.536084,0.744900,0.521005,0.341818,137.509998,141.953914
4,866a4a05fffffff,"{""type"":""Point"",""coordinates"":[36.293182542907...",235.258474,6157262,2.996913,16.144897,13.738791,11.077046,11.728739,10.285879,...,2.326510,2.513555,15.439214,1.408177,1.293108,0.627072,0.438593,0.287750,115.758746,119.499727


In [14]:
df['geometry'] = df['geometry'].apply(lambda geom: shape(json.loads(geom)))
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf

,hex_id,geometry,sum_pop_2020,ogc_fid,sum_pop_f_0_2020,sum_pop_f_10_2020,sum_pop_f_15_2020,sum_pop_f_1_2020,sum_pop_f_20_2020,sum_pop_f_25_2020,...,sum_pop_m_50_2020,sum_pop_m_55_2020,sum_pop_m_5_2020,sum_pop_m_60_2020,sum_pop_m_65_2020,sum_pop_m_70_2020,sum_pop_m_75_2020,sum_pop_m_80_2020,sum_pop_m_2020,sum_pop_f_2020
0,866a4a017ffffff,POINT (36.07869 5.11252),676.912804,6157254,8.924362,49.524105,36.898296,32.985901,28.963400,33.645473,...,6.732327,4.131749,53.380043,3.240541,2.475817,0.702667,0.399492,0.129285,336.082180,340.830624
1,866a4a01fffffff,POINT (36.13482 5.09047),347.182722,6157255,4.515680,24.773327,19.462585,16.690670,15.832314,16.429201,...,3.445144,2.752507,25.548544,1.827761,1.524122,0.585427,0.381080,0.209030,171.759225,175.423497
2,866a4a04fffffff,POINT (36.33929 5.11542),285.943490,6157260,3.642580,19.623219,16.698730,13.463530,14.255625,12.501910,...,2.827743,3.055085,18.765499,1.711561,1.571700,0.762171,0.533085,0.349744,140.698267,145.245222
3,866a4a057ffffff,POINT (36.23708 5.10296),279.463912,6157261,3.560037,19.178549,16.320332,13.158442,13.932587,12.218613,...,2.763665,2.985856,18.340267,1.672776,1.536084,0.744900,0.521005,0.341818,137.509998,141.953914
4,866a4a05fffffff,POINT (36.29318 5.0809),235.258474,6157262,2.996913,16.144897,13.738791,11.077046,11.728739,10.285879,...,2.326510,2.513555,15.439214,1.408177,1.293108,0.627072,0.438593,0.287750,115.758746,119.499727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25801,86969ed97ffffff,POINT (34.80875 -3.47225),196.448391,8710025,4.108127,14.176966,11.835453,15.015658,8.398741,6.313566,...,1.962373,1.294143,16.514441,1.084920,0.696495,0.607390,0.353109,0.295762,96.252689,100.195702
25802,86969ed9fffffff,POINT (34.75041 -3.44944),296.740893,8710026,6.205442,21.414711,17.877789,22.681580,12.686538,9.536821,...,2.964221,1.954840,24.945534,1.638802,1.052075,0.917481,0.533381,0.446756,145.392433,151.348460
25803,86969eda7ffffff,POINT (34.77046 -3.5715),154.818453,8710027,3.237562,11.172684,9.327369,11.833648,6.618940,4.975640,...,1.546521,1.019898,13.014819,0.855011,0.548899,0.478677,0.278281,0.233086,75.855508,78.962945
25804,86969edafffffff,POINT (34.7121 -3.54868),157.875350,8710028,3.301488,11.393290,9.511538,12.067303,6.749631,5.073884,...,1.577057,1.040035,13.271797,0.871894,0.559737,0.488128,0.283775,0.237688,77.353279,80.522071


In [15]:
# Define custom breaks and corresponding RGBA colors
breaks = [0, 1, 1000, 10000, 50000, 100000, 200000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 255],  # Light gray for 0
    [255, 255, 0, 255],    # Yellow for 1-1000
    [255, 165, 0, 255],    # Orange for 1000-10000
    [255, 0, 0, 255],      # Red for 10000-50000
    [128, 0, 128, 255],    # Purple for 50000-100000
    [0, 0, 255, 255],      # Blue for 100000-200000
    [0, 0, 139, 255],      # Dark blue for 200000+
])

# Function to assign colors based on custom bins
def assign_color(value, breaks, colors):
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i]
    return colors[-1]  # In case value exceeds all breaks

# Map sum_pop_2020 values to colors using the custom function
gdf['color'] = gdf["sum_pop_2020"].apply(lambda x: assign_color(x, breaks, colors))
colors = np.uint8(gdf['color'].tolist())

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=colors)

m = Map(layer)
m

Map(custom_attribution='', layers=(ScatterplotLayer(get_fill_color=arro3.core.ChunkedArray<FixedSizeList(Field…